In [1]:
import os
import re
import pickle
import datetime
import pandas as pd
import numpy as np

from _html_parser import ParsingDataPrepare, TableConstant

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

In [2]:
%load_ext autoreload
%autoreload 2

#### Choose the date for dirs and files

In [5]:
# Start from more time ('26_05_2022' -'27_03_2021'), ('26_03_2021' - '03_11_2013), ('02_11_2013' - '10_07_2005')
dir_data = dir_data_start = '26_03_2021'
dir_data_end = '03_11_2013'

In [6]:
with open('pickle_files/new_events/'+ dir_data +'/df_'+ dir_data_start +'_to_'+ dir_data_end  +'_games', 'rb') as f:
    df_games = pickle.load(f)

In [8]:
df_games

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue
0,1458512,Segunda División,707,306,26.03.2021 17:00,Sporting Gijón - Alcorcón,0:0,top
1,1458506,Segunda División,707,306,26.03.2021 19:00,Castellon - Espanyol,1:3,top
2,1459331,First League,435,306,26.03.2021 11:00,Mykolaiv - Alians Lypova Dolyna,2:0,sec
3,1459330,First League,435,306,26.03.2021 11:00,Polessya - Krystal Kherson,1:1,sec
4,1459334,First League,435,306,26.03.2021 12:00,Kremin - Hirnyk-Sport,1:1,sec
...,...,...,...,...,...,...,...,...
480417,14362043,Oberliga,428,38,09.11.2013 14:00,Jeddeloh - Osnabruck II,3:3,thr
480418,15052857,Premier League,670,3,06.11.2013 09:00,Karachi Port Trust - Khan Research Laboratories,0:2,thr
480419,15052851,Premier League,670,3,04.11.2013 09:00,Pakistan Navy - WAPDA FC,0:0,thr
480420,15052852,Premier League,670,3,04.11.2013 09:00,Afghan FC - Pakistan Army,0:0,thr


In [9]:
df_games['f_team'] = df_games.game_title.map(lambda x: str(x).split(' - ')[0])
df_games['s_team'] = df_games.game_title.map(lambda x: str(x).split(' - ')[1])

In [10]:
df_games.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,s_team
0,1458512,Segunda División,707,306,26.03.2021 17:00,Sporting Gijón - Alcorcón,0:0,top,Sporting Gijón,Alcorcón
1,1458506,Segunda División,707,306,26.03.2021 19:00,Castellon - Espanyol,1:3,top,Castellon,Espanyol
2,1459331,First League,435,306,26.03.2021 11:00,Mykolaiv - Alians Lypova Dolyna,2:0,sec,Mykolaiv,Alians Lypova Dolyna


#### Add competitions data 

In [11]:
with open('pickle_files/new_events/'+ dir_data +'/df_comp_tabs_'+ dir_data_start +'_all_lgs', 'rb') as f:
    df_comp_tables = pickle.load(f)

In [12]:
df_comp_tables.rename(columns={'team_name':'f_team'}, inplace=True)

In [13]:
# Duble column for get data for each team
df_comp_tables['s_team'] = df_comp_tables.f_team

In [14]:
df_comp_tables.head()

,f_team,team_id,pos,pld,w,d,l,gf,ga,+/-,pts,comp_id,season_id,s_team
0,Brazil U23,10614,1,3,1,2,0,4,0,+4,5,1016,179,Brazil U23
1,Denmark U23,10611,2,3,1,1,1,1,4,-3,4,1016,179,Denmark U23
2,Iraq U23,10610,3,3,0,3,0,1,1,0,3,1016,179,Iraq U23
3,South Africa U23,10615,4,3,0,2,1,1,2,-1,2,1016,179,South Africa U23
4,Nigeria U23,10621,1,3,2,0,1,6,6,0,6,1016,179,Nigeria U23


In [15]:
df_fcom = pd.merge(left=df_games, right=df_comp_tables[['f_team','comp_id','season_id','pos', 'pts']], on=['f_team','comp_id','season_id'], how='left')
df_fcom.rename(columns={'pos':'pos_f', 'pts':'pts_f'}, inplace=True)

In [16]:
df_scom = pd.merge(left=df_games, right=df_comp_tables[['s_team','comp_id','season_id','pos', 'pts']], on=['s_team','comp_id','season_id'], how='left')
df_scom.rename(columns={'pos':'pos_s', 'pts':'pts_s'}, inplace=True)

In [17]:
df_games_tb = pd.merge(df_fcom, df_scom[['game_id', 'pos_s','pts_s']], how='left', left_on='game_id', right_on='game_id')

In [18]:
df_games_tb.pos_f = pd.to_numeric(df_games_tb.pos_f, errors='coerce')
df_games_tb.pos_s = pd.to_numeric(df_games_tb.pos_s, errors='coerce')
df_games_tb.pts_f = pd.to_numeric(df_games_tb.pts_f, errors='coerce')
df_games_tb.pts_s = pd.to_numeric(df_games_tb.pts_s, errors='coerce')

In [19]:
df_games_tb['posf_poss'] = df_games_tb[['pos_f', 'pos_s']].apply(lambda x: str(x[0]).replace('.0', '') + ' - ' + str(x[1]).replace('.0', '') \
                                                                 if pd.notna(x[0]) and pd.notna(x[1]) else None, axis=1) 
df_games_tb['ptsf_ptss'] = df_games_tb[['pts_f', 'pts_s']].apply(lambda x: str(x[0]).replace('.0', '') + ' - ' + str(x[1]).replace('.0', '') \
                                                                 if pd.notna(x[0]) and pd.notna(x[1]) else None, axis=1) 

In [20]:
df_games_tb.drop(columns=['s_team'], inplace=True)

In [21]:
df_games_tb.drop_duplicates(subset=['game_id'], inplace=True)

In [22]:
df_games_tb.reset_index(drop=True, inplace=True)

In [23]:
df_games_tb.posf_poss.value_counts(dropna=False).head(4)

None     66416
1 - 2     3754
2 - 1     3527
3 - 1     3476
Name: posf_poss, dtype: int64

In [24]:
df_games_tb.posf_poss.value_counts(dropna=False).tail(4)

44 - 51    1
53 - 34    1
37 - 24    1
39 - 8     1
Name: posf_poss, dtype: int64

In [25]:
df_comp_tables.pos = pd.to_numeric(df_comp_tables.pos, errors='coerce')
df_comp_tables.pts = pd.to_numeric(df_comp_tables.pts, errors='coerce')

df_max_pos = df_comp_tables.groupby(['comp_id', 'season_id'])['pos'].max().to_frame().reset_index()  
df_max_pts = df_comp_tables.groupby(['comp_id', 'season_id'])['pts'].max().to_frame().reset_index() 

df_max_pos.rename(columns={'pos':'max_pos'}, inplace=True)
df_max_pts.rename(columns={'pts':'max_pts'}, inplace=True)

In [26]:
df_max_pos.head(3)

,comp_id,season_id,max_pos
0,1016,179,4
1,1019,173,4
2,1019,294,4


In [27]:
df_max_pts.head(3)

,comp_id,season_id,max_pts
0,1016,179,7
1,1019,173,16
2,1019,294,16


In [28]:
df_games_tb = df_games_tb.merge(df_max_pos, how='left', left_on=['comp_id', 'season_id'],  right_on=['comp_id', 'season_id'])
df_games_tb = df_games_tb.merge(df_max_pts, how='left', left_on=['comp_id', 'season_id'],  right_on=['comp_id', 'season_id'])

In [29]:
df_games_tb['fs_pos_max'] = df_games_tb[['posf_poss', 'max_pos']].apply(lambda x: str(x[0])+ ', ' + str(x[1]).replace('.0', '') \
                                                                                                                  if x[0] != None else None, axis=1)
df_games_tb['fs_pts_max'] = df_games_tb[['ptsf_ptss', 'max_pts']].apply(lambda x: str(x[0])+ ', ' + str(x[1]).replace('.0', '') \
                                                                                                                  if x[0] != None else None, axis=1)

In [30]:
df_games_tb.drop(columns=['posf_poss', 'ptsf_ptss', 'max_pts'], inplace=True)

#### Concatinate dataframes with best and tables position coefficients

In [31]:
df_power_coefs = df_games_tb[df_games_tb.max_pos >= 8].copy().reset_index(drop=True)
df_power_coefs.shape[0]

412663

In [32]:
df_power_coefs.max_pos.value_counts(dropna=False)

20.0    68409
16.0    57719
18.0    55526
12.0    39041
24.0    33680
        ...  
28.0      378
26.0      325
27.0      257
29.0      255
34.0       61
Name: max_pos, Length: 25, dtype: int64

In [33]:
df_power_coefs.drop(columns=['max_pos'], inplace=True)

In [34]:
print('shape - ', df_power_coefs.shape[0])
df_power_coefs.tail(10)

shape -  412663


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max
412653,14359601,Oberliga,428,38,09.11.2013 12:30,Rot-WeiY Hadamar - FSV Frankfurt II,1:1,thr,Rot-WeiY Hadamar,6.0,55.0,5.0,56.0,"6 - 5, 20","55 - 56, 96"
412654,14365780,Oberliga,428,38,09.11.2013 12:30,Hauenstein - Burgbrohl,0:5,thr,Hauenstein,3.0,55.0,10.0,43.0,"3 - 10, 20","55 - 43, 96"
412655,14365782,Oberliga,428,38,09.11.2013 12:30,Rochling Volklingen - Hertha Wiesbach,0:3,thr,Rochling Volklingen,14.0,40.0,8.0,47.0,"14 - 8, 20","40 - 47, 96"
412656,14362982,Oberliga,428,38,09.11.2013 13:00,Malchower SV - Schоnberg,1:1,thr,Malchower SV,13.0,33.0,11.0,36.0,"13 - 11, 20","33 - 36, 96"
412657,14365783,Oberliga,428,38,09.11.2013 13:00,Mehring - Salmrohr,2:0,thr,Mehring,16.0,39.0,2.0,60.0,"16 - 2, 20","39 - 60, 96"
412658,14362043,Oberliga,428,38,09.11.2013 14:00,Jeddeloh - Osnabruck II,3:3,thr,Jeddeloh,10.0,41.0,4.0,46.0,"10 - 4, 20","41 - 46, 96"
412659,15052857,Premier League,670,3,06.11.2013 09:00,Karachi Port Trust - Khan Research Laboratories,0:2,thr,Karachi Port Trust,8.0,40.0,NaN,NaN,None,None
412660,15052851,Premier League,670,3,04.11.2013 09:00,Pakistan Navy - WAPDA FC,0:0,thr,Pakistan Navy,12.0,35.0,3.0,63.0,"12 - 3, 16","35 - 63, 69"
412661,15052852,Premier League,670,3,04.11.2013 09:00,Afghan FC - Pakistan Army,0:0,thr,Afghan FC,10.0,36.0,5.0,48.0,"10 - 5, 16","36 - 48, 69"
412662,15052853,Premier League,670,3,04.11.2013 09:00,Muslim - Khan Research Laboratories,1:1,thr,Muslim,7.0,44.0,NaN,NaN,None,None


In [36]:
# Stop

#### Power coefficient  ' * ' :  2

#### Power coefficient sum: -13 pts_f  (home_team = fav) < home_team_pts_f < + 13 pts_f  (home_team = pre)

In [37]:
# pow_coef = 2
# df_power_coefs['f_pow_pts'] = df_power_coefs[['pts_f', 'pts_s']].apply(lambda x: ParsingDataPrepare.roles_determinate([x[0], x[1]], pow_coef, 'max_point'), axis=1)

In [38]:
sum_coef = 13
df_power_coefs['f_pow_pts'] = df_power_coefs[['pts_f', 'pts_s']].apply(lambda x: ParsingDataPrepare.roles_determinate([x[0], x[1]], sum_coef, 'sum_point'), axis=1)

In [39]:
df_power_coefs.f_pow_pts.value_counts(dropna=False)

Neu    215965
Fav     98585
Pre     98113
Name: f_pow_pts, dtype: int64

In [40]:
df_power_coefs['f_pow'] = df_power_coefs.f_pow_pts

In [41]:
df_power_coefs.drop(columns='f_pow_pts', inplace=True)

In [42]:
df_fav_pre = df_power_coefs[df_power_coefs.f_pow.isin(['Fav', 'Pre'])].copy()

In [43]:
# Stop

#### Set description

In [44]:
# 'pow_coef_2'
df_fav_pre['desc'] = 'sum_coef_13'

In [45]:
df_fav_pre.reset_index(drop=True, inplace=True)

In [46]:
print('shape - ', df_fav_pre.shape[0])
df_fav_pre.tail(3)

shape -  196698


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc
196695,14354938,Oberliga,428,38,09.11.2013 12:30,Bremerhaven - Vatan Sport,6:1,thr,Bremerhaven,9.0,41.0,14.0,22.0,"9 - 14, 20","41 - 22, 96",Fav,sum_coef_13
196696,14365783,Oberliga,428,38,09.11.2013 13:00,Mehring - Salmrohr,2:0,thr,Mehring,16.0,39.0,2.0,60.0,"16 - 2, 20","39 - 60, 96",Pre,sum_coef_13
196697,15052851,Premier League,670,3,04.11.2013 09:00,Pakistan Navy - WAPDA FC,0:0,thr,Pakistan Navy,12.0,35.0,3.0,63.0,"12 - 3, 16","35 - 63, 69",Pre,sum_coef_13


In [47]:
df_for_cities = df_fav_pre.copy()

In [48]:
df_for_cities.shape

(196698, 17)

In [50]:
# Stop

#### Remove previous game_ids with 'pow_coef_2' description if exist.

In [53]:
print('dir_data - ', dir_data)
with open('pickle_files/aspected_files/df_with_aspects_' + dir_data + '_f_pow_2', 'rb') as f:
    df_with_asps = pickle.load(f)

dir_data -  26_03_2021


In [54]:
df_with_asps.shape

(56017, 27)

In [55]:
df_for_cities = df_for_cities[~df_for_cities.game_id.isin(df_with_asps.game_id)].copy().reset_index(drop=True)
print('shape - ', df_for_cities.shape)
df_for_cities.tail(3)

shape -  (142454, 17)


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc
142451,14354938,Oberliga,428,38,09.11.2013 12:30,Bremerhaven - Vatan Sport,6:1,thr,Bremerhaven,9.0,41.0,14.0,22.0,"9 - 14, 20","41 - 22, 96",Fav,sum_coef_13
142452,14365783,Oberliga,428,38,09.11.2013 13:00,Mehring - Salmrohr,2:0,thr,Mehring,16.0,39.0,2.0,60.0,"16 - 2, 20","39 - 60, 96",Pre,sum_coef_13
142453,15052851,Premier League,670,3,04.11.2013 09:00,Pakistan Navy - WAPDA FC,0:0,thr,Pakistan Navy,12.0,35.0,3.0,63.0,"12 - 3, 16","35 - 63, 69",Pre,sum_coef_13


#### Add cities and countries. Got ids from 'soccer365' site. Check last created file.

In [56]:
with open('pickle_files/cities_countries/df_countries_champ_ids_created_20_07_22', 'rb') as f:
    df_countries = pickle.load(f)

In [57]:
df_countries.drop_duplicates(subset=['comp_id', 'country'], inplace=True)
df_countries = df_countries[~df_countries.country.isin([''])]
df_countries.reset_index(drop=True, inplace=True)

In [58]:
print('shape - ', df_countries.shape)
df_countries

shape -  (385, 2)


,comp_id,country
0,12,England
1,13,Russia
2,14,Ukraine
3,15,Italy
4,16,Spain
...,...,...
380,1199,nd
381,526,nd
382,780,nd
383,2062,Mexico


In [59]:
str(set(df_for_cities[~df_for_cities.comp_id.isin(df_countries.comp_id)].comp_id))

'set()'

In [ ]:
# '467':'Australia', 538':'Canada', '1695':'Kenya', '719':'Trinidad and Tobago', '583':'Gabon', '702':'Slovakia', '663':'New Zealand', '1074':'South Africa'
# '1767':'USA', '594':'Haiti', '608':'Indonesia', '497':'Brazil', '696':'Senegal', '490':'Bhutan', '662':'Namibia', '480':'Barbados', '1078':'Guinea'
# '1081':'Burkina Faso', '573':'Faroe Islands', '2062': 'Mexico'

In [ ]:
# df_countries = df_countries.append({'comp_id':'2076', 'country':'nd'}, ignore_index=True)

In [ ]:
# file = open('pickle_files/cities_countries/df_countries_champ_ids_created_20_07_22', 'wb')
# pickle.dump(df_countries, file)  
# file.close()

In [60]:
print('shape - ', df_for_cities.shape[0])
df_for_cities.tail(3)

shape -  142454


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc
142451,14354938,Oberliga,428,38,09.11.2013 12:30,Bremerhaven - Vatan Sport,6:1,thr,Bremerhaven,9.0,41.0,14.0,22.0,"9 - 14, 20","41 - 22, 96",Fav,sum_coef_13
142452,14365783,Oberliga,428,38,09.11.2013 13:00,Mehring - Salmrohr,2:0,thr,Mehring,16.0,39.0,2.0,60.0,"16 - 2, 20","39 - 60, 96",Pre,sum_coef_13
142453,15052851,Premier League,670,3,04.11.2013 09:00,Pakistan Navy - WAPDA FC,0:0,thr,Pakistan Navy,12.0,35.0,3.0,63.0,"12 - 3, 16","35 - 63, 69",Pre,sum_coef_13


In [61]:
df_for_cities = df_for_cities.merge(df_countries, how='left', left_on='comp_id', right_on='comp_id')

In [62]:
df_for_cities.shape

(142454, 18)

In [63]:
df_for_cities.country.value_counts(dropna=False)

Spain      22120
England    17484
Italy      11549
Germany    11146
Turkey      4594
           ...  
Cuba          12
Namibia       10
Aruba          5
Canada         3
Bhutan         2
Name: country, Length: 139, dtype: int64

In [ ]:
# Stop

#### Add new data. Check last created file.

In [64]:
with open('pickle_files/teams_data/df_tmd_main_created_22_07_2022', 'rb') as f:
    df_teams_data = pickle.load(f)

In [65]:
print('shape - ', df_teams_data.shape[0])
df_teams_data.tail(3)

shape -  9217


,f_team,team_id,country,city
9214,Song Lam Nghe An,11391,Vietnam,Vinh
9215,Heidelberg United,12529,Australia,Melbourne
9216,Mladost Medoševac,NaN,Serbia,Medoševac


In [66]:
df_teams_data[df_teams_data.duplicated(subset=['f_team', 'country'])]

,f_team,team_id,country,city


In [67]:
df_teams_data.rename(columns={'team':'f_team'}, inplace=True)

In [68]:
print('shape - ', df_for_cities.shape[0])
df_for_cities = df_for_cities.merge(df_teams_data, how='left', left_on=['f_team', 'country'], right_on=['f_team', 'country'])
print('shape - ', df_for_cities.shape[0])

shape -  142454
shape -  142454


In [69]:
df_for_cities.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,f_pow,desc,country,team_id,city
0,1458512,Segunda División,707,306,26.03.2021 17:00,Sporting Gijón - Alcorcón,0:0,top,Sporting Gijón,7.0,65.0,17.0,48.0,"7 - 17, 22","65 - 48, 82",Fav,sum_coef_13,Spain,295,Gijón
1,1458506,Segunda División,707,306,26.03.2021 19:00,Castellon - Espanyol,1:3,top,Castellon,21.0,41.0,1.0,82.0,"21 - 1, 22","41 - 82, 82",Pre,sum_coef_13,Spain,8183,Castellón de la Plana
2,1459334,First League,435,306,26.03.2021 12:00,Kremin - Hirnyk-Sport,1:1,sec,Kremin,15.0,24.0,9.0,38.0,"15 - 9, 16","24 - 38, 68",Pre,sum_coef_13,Ukraine,8532,Kremenchuk


In [70]:
df_for_cities = df_for_cities[~df_for_cities.country.isin(['nd'])].reset_index(drop=True).copy()

In [71]:
df_for_cities.city.value_counts(dropna=False)[:3]

NaN             8722
Buenos Aires    1035
London          1013
Name: city, dtype: int64

#### Save this file and uncomment rows after manual adding cities and teams data

In [ ]:
# Stop

In [ ]:
# df_for_cities = df_for_cities[~df_for_cities.city.isna()].reset_index(drop=True).copy()

In [ ]:
# df_for_cities = df_for_cities[~df_for_cities.city.isin(['nd'])].reset_index(drop=True).copy()

In [ ]:
# df_for_cities

In [ ]:
# df_for_geo_coords = df_for_cities.copy()
# print('shape - ', df_for_geo_coords.shape[0])
# df_for_geo_coords.tail(3)

In [ ]:
# file = open('pickle_files/df_for_geo_coords_26_05_2022_sec', 'wb')
# pickle.dump(df_for_geo_coords, file)  
# file.close()

#### For find teams with na cities

In [72]:
df_na_cities = pd.DataFrame(df_for_cities[['f_team', 'country']][df_for_cities.city.isna()])

In [73]:
df_na_cities = df_na_cities.drop_duplicates(subset=['f_team', 'country'], keep='first').reset_index(drop=True)

In [74]:
df_na_cities

,f_team,country
0,Fortuna Becicherecu Mic,Romania
1,Concordia Chiajna II,Romania
2,Flacara,Romania
3,Phrae United,Thailand
4,Nakhon Pathom,Thailand
...,...,...
2278,Lyallpur,Pakistan
2279,Pak Afghan Clearing,Pakistan
2280,Pakistan Airlines,Pakistan
2281,Habib Bank,Pakistan


In [75]:
df_teams_counts_na = df_for_cities[df_for_cities.city.isna()].f_team.value_counts()

In [76]:
numb = 0

In [77]:
len(df_teams_counts_na[df_teams_counts_na >= numb])

2278

In [78]:
df_teams_counts_na[df_teams_counts_na >= numb].values.sum()

8722

In [79]:
df_teams_counts_na

Dungannon                   43
Thamesmead Town             41
Paterna                     36
Colwyn Bay                  35
Babelsberg 03               31
                            ..
Serrano PB                   1
Sidi Bouzid                  1
Strasbourg II                1
Eastern Suburbs Brisbane     1
Waltersdorf                  1
Name: f_team, Length: 2278, dtype: int64

In [80]:
df_teams_counts_na_numb = df_teams_counts_na[df_teams_counts_na >= numb].index

In [81]:
df_teams_counts_na_numb

Index(['Dungannon ', 'Thamesmead Town', 'Paterna', 'Colwyn Bay',
       'Babelsberg 03 ', 'Groybardorf', 'Quintanar del Rey', 'Dnepr Smolensk',
       'Crevillente', 'Toronto FC II',
       ...
       'Doze', 'Bilzerse Waltwilder', 'Olympique de Mandji',
       'AS Pierrots Vauban Strasbourg', 'FC Lubumbashi sport', 'Serrano PB',
       'Sidi Bouzid', 'Strasbourg II', 'Eastern Suburbs Brisbane',
       'Waltersdorf'],
      dtype='object', length=2278)

In [82]:
df_teams_cities_na = df_na_cities[df_na_cities.f_team.isin(df_teams_counts_na_numb)].reset_index(drop=True)

In [83]:
df_teams_cities_na

,f_team,country
0,Fortuna Becicherecu Mic,Romania
1,Concordia Chiajna II,Romania
2,Flacara,Romania
3,Phrae United,Thailand
4,Nakhon Pathom,Thailand
...,...,...
2278,Lyallpur,Pakistan
2279,Pak Afghan Clearing,Pakistan
2280,Pakistan Airlines,Pakistan
2281,Habib Bank,Pakistan


In [84]:
dir_data

'26_03_2021'

In [85]:
pow_coef_desc = df_fav_pre['desc'][0]
pow_coef_desc

'sum_coef_13'

In [87]:
# file = open('pickle_files/df_teams_cities_na_' + dir_data +'_'+ pow_coef_desc, 'wb')
# pickle.dump(df_teams_cities_na, file)  
# file.close()